In [1]:
import pandas as pd 

In [ ]:
pip install fastparquet

In [ ]:
df = pd.read_parquet('./Top10_Centroide_Kmean.parquet')

# Display the DataFrame
print(df.head())

In [5]:
df.head()

,file,message,parsed_email,Message-ID,X-FileName,Body,final_body,embedding,spam,From,KMeans_Labels,hdbscan_cluster,Distance_to_Centroid
0,stokley-c/chris_stokley/murray/35.,Message-ID: <3731370.1075858521444.JavaMail.ev...,{'Message-ID': '<3731370.1075858521444.JavaMai...,<3731370.1075858521444.JavaMail.evans@thyme>,"Stokley, Chris (Non-Privileged).pst","\nLyman,\n\nYea, he should be able to cover mo...","Lyman, Yea, able cover concerns inputs. Sorry ...","[-0.714, -0.0787, -0.6055, -0.4688, 0.12354, 0...",0,p..o'neil@enron.com,2,293,9.912337
1,watson-k/e_mail_bin/29.,Message-ID: <521887.1075860893691.JavaMail.eva...,{'Message-ID': '<521887.1075860893691.JavaMail...,<521887.1075860893691.JavaMail.evans@thyme>,kwatson (Non-Privileged).pst,"\nKim-\n\nAs I had mentioned, I'd like to find...","Kim- mentioned, I'd like find time could meet ...","[-0.742, -0.545, -0.4968, -0.3433, 0.2354, -0....",0,morgan.gottsponer@enron.com,2,293,10.490339
2,ward-k/sent_items/386.,Message-ID: <3957896.1075852874589.JavaMail.ev...,{'Message-ID': '<3957896.1075852874589.JavaMai...,<3957896.1075852874589.JavaMail.evans@thyme>,KWARD (Non-Privileged).pst,"\nArt, \n\nIt looks like I won't be able to ma...","Art, looks like able make it. meetings set see...","[-0.7866, 0.03152, -1.187, -0.5903, -0.221, -0...",0,houston <.ward@enron.com>,2,293,10.627373
3,dasovich-j/sent_items/474.,Message-ID: <1897265.1075851621982.JavaMail.ev...,{'Message-ID': '<1897265.1075851621982.JavaMai...,<1897265.1075851621982.JavaMail.evans@thyme>,"Dasovich, Jeff (Non-Privileged).pst","\nGreat. Figured they'd contacted them, but w...","Great. Figured they'd contacted them, wanted m...","[-0.923, -0.58, 0.4043, -0.267, 0.1324, -0.502...",0,jeff.dasovich@enron.com,2,293,10.745469
4,blair-l/deleted_items/212.,Message-ID: <12027492.1075853060996.JavaMail.e...,{'Message-ID': '<12027492.1075853060996.JavaMa...,<12027492.1075853060996.JavaMail.evans@thyme>,LBLAIR (Non-Privileged).pst,"\nLynn, I guess there were two handouts that I...","Lynn, guess two handouts receive yesterday. Wo...","[-0.02414, -0.891, -0.1721, 0.00828, 0.982, -0...",0,deborah.cappiello@enron.com,2,293,10.902901


In [1]:
# grouped_emails = df.groupby("kmeans_cluster").agg({
#     "final_body": list,
#     "Message-ID": list
# }).reset_index()
# grouped_emails.iloc[0]

# Clusters Labelization

In [ ]:
import pandas as pd
import json
from langchain_openai import ChatOpenAI
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configuration de l'API
url = "https://aristote-dispatcher.mydocker-run-vd.centralesupelec.fr/v1"
llm = ChatOpenAI(
    base_url=url,
    model="casperhansen/llama-3-70b-instruct-awq",
    api_key="nothing"  # Remplacez "nothing" par votre clé API si nécessaire
)

# Prompt système
system_prompt = """
You are an expert in text classification and analysis. Your task is to analyze the provided clusters of emails and determine:
1. The main topic of the emails within a cluster. (Generalize the topic to a global category such as: market strategy, personal, energy market, IT, meeting planning, finance, legal, human ressources, contracts, fraude etc.)
2. Note that for each cluster I will provide you 10 emails that represents the top nearest emails to the centroid of the cluster, so they are a representation of a larger set of email.
3. Avoid assigning the same general topic to multiple clusters whenever possible. Strive to be specific in identifying the topic while ensuring it remains aligned with a general category.
4. If no topic is detected return "common_topic": "N/A", and explanatory note for better understanding
Return the results in the following JSON format:
{
    "kmeans_cluster": <kmeans_cluster>,
    "common_topic": "<description>",
    "Note": "<additional note to provide better context>"
}

Make sure your response is strictly valid JSON and that the classification is accurate based on the above instructions.
"""

# Fonction pour traiter un cluster d'e-mails
def process_cluster(row):
    kmeans_cluster = row["KMeans_Labels"]
    emails = row["final_body"]

    batch_text = "\n\n".join([f"Email {i+1}: {email}" for i, email in enumerate(emails)])
    batch_prompt = f"The following emails belong to cluster {kmeans_cluster}:{batch_text}"

    messages = [
        ("system", system_prompt),
        ("human", batch_prompt),
    ]
    #print(f"Sending payload for cluster {kmeans_cluster}:\n", batch_prompt)

    try:
        response = llm.invoke(messages)
        response_content = response.content if hasattr(response, "content") else None

        if response_content:
            start_idx = response_content.find("{")
            end_idx = response_content.rfind("}")
            if start_idx != -1 and end_idx != -1:
                json_part = response_content[start_idx:end_idx + 1]
                try:
                    result = json.loads(json_part)
                    result["KMeans_Labels"] = kmeans_cluster
                except json.JSONDecodeError:
                    result = {
                        "kmeans_cluster": kmeans_cluster,
                        "common_topic": "N/A",
                        "Note": "Failed to decode JSON."
                    }
            else:
                result = {
                    "kmeans_cluster": kmeans_cluster,
                    "common_topic": "N/A",
                    "Note": "No JSON object found in response."
                }
        else:
            result = {
                "kmeans_cluster": kmeans_cluster,
                "common_topic": "N/A",
                "Note": "Empty response."
            }

    except Exception as e:
        result = {
            "kmeans_cluster": kmeans_cluster,
            "common_topic": "N/A",
            "Note": f"Exception occurred: {str(e)}"
        }

    return result

# Grouper les e-mails par cluster
grouped_emails = df.groupby("KMeans_Labels").agg({
    "final_body": list
}).reset_index()

# Liste pour stocker les résultats
results = []

# Utilisation de ThreadPoolExecutor pour le multithreading
with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_cluster = {executor.submit(process_cluster, row): row for _, row in grouped_emails.iterrows()}
    for future in tqdm(as_completed(future_to_cluster), total=len(grouped_emails), desc="Processing clusters"):
        try:
            result = future.result()
            results.append(result)
        except Exception as e:
            print(f"Error processing cluster: {e}")

# Convertir les résultats en DataFrame
results_df = pd.DataFrame(results)

# Ajouter les colonnes "common_topic" et "is_legal" au DataFrame original
df = df.merge(results_df[["KMeans_Labels", "common_topic"]], on="KMeans_Labels", how="left")

# Sauvegarder les résultats dans des fichiers
output_csv_file = "clusters_classification_with_topics.csv"
df.to_csv(output_csv_file, index=False)

output_json_file = "clusters_classification_with_topics.json"
with open(output_json_file, "w") as f:
    json.dump(results, f, indent=4)

print(f"Analysis results have been saved to {output_json_file} and {output_csv_file}.")

In [2]:
import pandas as pd 
df_global = pd.read_parquet('./KMeans_HDBSCAN_Clustering.parquet')

In [3]:
df = pd.read_csv('./clusters_classification_with_topics.csv')

In [4]:
df_final = df_global.merge(df[["KMeans_Labels", "common_topic"]], on="KMeans_Labels",how="left")


In [ ]:
df_final.to_parquet('final_df_merged_with_topic.parquet')